<div >
<img src = "../banner.png" />
</div>

# Clase Final Sobre Sistemas de Recomendación 

Este *cuaderno* trata sobre sistemas de recomendación basados en ubicación y algunas maneras de crear sistemas mixtos, donde podamos usar la combinación de varios metodos vistos en clases anteriores. El objetivo del *ciaderno* es que usted vea un caso simple de uso de estos sistemas para entender sus usos y escabilidad, implemente los algoritmos basados en ubicación y que sea capaz de reconocer sus características, funcionamiento, y  desarrollarlos en `Python`.

In [1]:
#cargamos librerias
import pandas as pd
import numpy as np

In [2]:
# Cargamos y visualizamos  los datos
ratings = pd.read_excel('/Users/juank/Dropbox/Mac/Desktop/RA_machine/edu_continua/L10_Recomendac_Mixtos/data/Ratings.xlsx')
ratings.head()

,restaurant_id,user_id,rating,comentarios
0,1,1,5.0,Excelente Servicio. La comida exquisita. Mejor...
1,1,2,4.0,"La comida deliciosa, el trato puede mejorar"
2,1,3,3.0,"Muy buena comida, pero mal servicio"
3,1,4,NaN,NaN
4,1,5,NaN,NaN


In [3]:
# Cargamos y visualizamos  los datos
usuarios = pd.read_excel('/Users/juank/Dropbox/Mac/Desktop/RA_machine/edu_continua/L10_Recomendac_Mixtos/data/Usuarios.xlsx')
usuarios

,user_id,nombre,latitud,longitud
0,1,Juan,4.652834,-74.054339
1,2,Maria,4.653065,-74.054058
2,3,Pablo,4.653195,-74.059075
3,4,Joaquin,4.653328,-74.055814
4,5,Martina,4.653515,-74.058822
5,6,Rosario,4.653647,-74.058792


In [4]:
# Cargamos y visualizamos  los datos
restaurants = pd.read_excel('/Users/juank/Dropbox/Mac/Desktop/RA_machine/edu_continua/L10_Recomendac_Mixtos/data/Restaurants.xlsx')
restaurants.head()

,restaurant_id,latitud,longitud,nombre,tipo
0,1,4.651420,-74.054638,Vegan Island,vegano
1,2,4.651475,-74.054732,La Verdura Inquieta,vegano
2,3,4.651530,-74.054834,El Pirata del sabor vegano,vegano
3,4,4.655471,-74.057151,Muuu said the happy cow,carne
4,5,4.655363,-74.057581,Paul's Mcarne Feast Palace,carne


In [5]:
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='restaurant_id')
r_matrix

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,NaN,NaN
2,4.0,3.0,4.0,NaN,NaN
3,3.0,2.0,3.0,NaN,NaN
4,NaN,NaN,NaN,4.0,5.0
5,NaN,NaN,NaN,4.0,5.0
6,4.0,3.0,4.0,2.0,4.0


# Filtrado Colaborativo Basado en Usuarios.

El filtrado colaborativo aprovecha el poder de la colaboración para generar recomendaciones. 


Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con 5 usuarios y 5 productos, en este caso restaurantes. 

El valor de la celda denota el rating que le dió cada usuario al ítem. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al restaurante $j$. 

### Medias

- Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario 

In [6]:
# Crearemos un promedio por restaurante
puntuac_promedio = ratings.groupby('restaurant_id').mean()
medias = pd.DataFrame(puntuac_promedio.iloc[:,1])
medias

,rating
restaurant_id,
1,4.000000
2,3.000000
3,4.000000
4,3.333333
5,4.666667


In [7]:
medias = medias.reset_index()
restaurants = restaurants.merge(medias, how='left', on='restaurant_id')
restaurants

,restaurant_id,latitud,longitud,nombre,tipo,rating
0,1,4.651420,-74.054638,Vegan Island,vegano,4.000000
1,2,4.651475,-74.054732,La Verdura Inquieta,vegano,3.000000
2,3,4.651530,-74.054834,El Pirata del sabor vegano,vegano,4.000000
3,4,4.655471,-74.057151,Muuu said the happy cow,carne,3.333333
4,5,4.655363,-74.057581,Paul's Mcarne Feast Palace,carne,4.666667


### Medias ponderadas

- Podemos hacer algo más sofisticado usando recomendaciones de usuarios similares?


- Podemos hacer una media ponderada


$$
r_{ur}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'r}}{\sum_{u',u'\neq u}sim(u,u')}
$$



- Es decir la predicción del rating del usuario $u$ para el restaurante $r$, $r_{ur}$, es la suma ponderada de los ratings de los otros usuarios ($u'$) a este restaurante, 


- Ponderado por cuán similares son los usuarios $u'$ a $u$. 


- Como medimos similitud?

    - Existen múltiples medidas de distancia que se utilizan para medir la similitud. 

    - La distancia de coseno, que suele ser la más utilizada en los sistemas de recomendación.

    - Matemáticamente

$$
coseno(x,y)=\frac{x.y'}{|x||y|}
$$


Es decir, es el cociente del producto punto, dividido por las normas de los vectores.

<center>
<img src = "figs/dist_cos.png" alt = "coseno" style = "width: 300px;"/>
</center>

- Si el ángulo es 0  de grados, entonces los vectores se solapan, y el coseno es igual a 1 
- Si el ángulo es 90 de grados, los vectores forman un angulo recto, y el coseno es igual a 0.
- Si el ángulo es 180 de grados, los vectores estan en sentido opuesto, y  el coseno es igual a -1.

In [8]:
# Creamos nuestra tabla dinamica de usuarios y restaurantes
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,0.0,0.0
2,4.0,3.0,4.0,0.0,0.0
3,3.0,2.0,3.0,0.0,0.0
4,0.0,0.0,0.0,4.0,5.0
5,0.0,0.0,0.0,4.0,5.0
6,4.0,3.0,4.0,2.0,4.0


In [9]:
# Importamos cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz resultante en un dataframe
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim

user_id,1,2,3,4,5,6
user_id,,,,,,
1,1.000000,0.999630,0.997241,0.000000,0.000000,0.819533
2,0.999630,1.000000,0.998891,0.000000,0.000000,0.819836
3,0.997241,0.998891,1.000000,0.000000,0.000000,0.818927
4,0.000000,0.000000,0.000000,1.000000,1.000000,0.559888
5,0.000000,0.000000,0.000000,1.000000,1.000000,0.559888
6,0.819533,0.819836,0.818927,0.559888,0.559888,1.000000


In [10]:
def cf_user_wmean(user_id, restaurant_id):
    
    # Primero verificamos si la película esta en la matriz
    if restaurant_id in r_matrix:
    
        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]
        
        
        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[restaurant_id]
        
         # Obtenemos los índicies de las películas sin rating 
        idx = m_ratings[m_ratings.isnull()].index
        
        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()
        
        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0
    
    return wmean_rating

In [11]:
cf_user_wmean(4,5)

4.78128417093154

Podemos observar que el puntaje ponderado que tiene Joaquin (usuario 4) para el restaurante Paul's Mcarne Feast Palace (restaurante 5) es de 4.7, ya que su usuario mas similar lo califico de 5 y su puntaje pesa mas que el del usuario 6.

In [12]:
sim_scores = cosine_sim[4]
sim_scores

user_id
1    0.000000
2    0.000000
3    0.000000
4    1.000000
5    1.000000
6    0.559888
Name: 4, dtype: float64

In [13]:
m_ratings =r_matrix[5]
m_ratings

user_id
1    NaN
2    NaN
3    NaN
4    5.0
5    5.0
6    4.0
Name: 5, dtype: float64

In [14]:
idx = m_ratings[m_ratings.isnull()].index
idx

Int64Index([1, 2, 3], dtype='int64', name='user_id')

In [15]:
sim_scores1 = sim_scores.drop(idx)

m_ratings = m_ratings.dropna()
        
# Calculamos la media ponderada
wmean_rating = np.dot(sim_scores1, m_ratings)/ sim_scores1.sum()
wmean_rating

4.78128417093154

### Transformación a geopandas

In [16]:
#Cargamos geopandas que es la librería a utilizar
import geopandas as gpd

In [17]:
restaurants = gpd.GeoDataFrame(restaurants, geometry=gpd.points_from_xy(restaurants.longitud, restaurants.latitud),crs=4326)
restaurants

,restaurant_id,latitud,longitud,nombre,tipo,rating,geometry
0,1,4.651420,-74.054638,Vegan Island,vegano,4.000000,POINT (-74.05464 4.65142)
1,2,4.651475,-74.054732,La Verdura Inquieta,vegano,3.000000,POINT (-74.05473 4.65147)
2,3,4.651530,-74.054834,El Pirata del sabor vegano,vegano,4.000000,POINT (-74.05483 4.65153)
3,4,4.655471,-74.057151,Muuu said the happy cow,carne,3.333333,POINT (-74.05715 4.65547)
4,5,4.655363,-74.057581,Paul's Mcarne Feast Palace,carne,4.666667,POINT (-74.05758 4.65536)


In [18]:
usuarios = gpd.GeoDataFrame(usuarios, geometry=gpd.points_from_xy(usuarios.longitud, usuarios.latitud),crs=4326)
usuarios

,user_id,nombre,latitud,longitud,geometry
0,1,Juan,4.652834,-74.054339,POINT (-74.05434 4.65283)
1,2,Maria,4.653065,-74.054058,POINT (-74.05406 4.65306)
2,3,Pablo,4.653195,-74.059075,POINT (-74.05908 4.65320)
3,4,Joaquin,4.653328,-74.055814,POINT (-74.05581 4.65333)
4,5,Martina,4.653515,-74.058822,POINT (-74.05882 4.65351)
5,6,Rosario,4.653647,-74.058792,POINT (-74.05879 4.65365)


In [19]:
import folium

map = folium.Map(location = [4.65283,-74.054339], tiles = "OpenStreetMap", zoom_start = 16)
# Otras opciones de tiles
#Stamen Terrain, Toner, and Watercolor


for i in range(0,len(restaurants)):
   folium.Marker(
      location=[restaurants.iloc[i]['latitud'], restaurants.iloc[i]['longitud']],
       popup=restaurants.iloc[i]['nombre'],
   ).add_to(map)

for i in range(0,len(usuarios)):
   folium.Marker(
      location=[usuarios.iloc[i]['latitud'], usuarios.iloc[i]['longitud']],
       popup=usuarios.iloc[i]['nombre'], icon=folium.Icon(color='red')
   ).add_to(map)
    

#Display el mapa
map

# Midiendo Distancias

In [20]:
usuarios.geometry.apply(lambda g: restaurants.distance(g))

<ipython-input-20-72883107f552>:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  usuarios.geometry.apply(lambda g: restaurants.distance(g))


,0,1,2,3,4
0,0.001445,0.001415,0.001395,0.003855,0.004112
1,0.001744,0.001727,0.001720,0.003919,0.004206
2,0.004779,0.004671,0.004556,0.002980,0.002633
3,0.002241,0.002146,0.002048,0.002526,0.002695
4,0.004679,0.004571,0.004455,0.002573,0.002226
5,0.004713,0.004604,0.004489,0.002454,0.002100


## La tierra no es plana !!

In [21]:
from IPython.display import IFrame
IFrame('https://gifs.com/gif/mercator-projection-y4xP7j', width=700, height=350)

# Calculamos las distancias

In [22]:
usuarios = usuarios.to_crs(3116)
restaurants = restaurants.to_crs(3116)
distancias = usuarios.geometry.apply(lambda g: restaurants.distance(g))
distancias

,0,1,2,3,4
0,159.843081,156.478825,154.303897,427.059596,455.637595
1,192.954635,191.067920,190.331335,434.237728,466.233130
2,529.994347,518.056803,505.298993,330.026024,291.469123
3,248.077661,237.486837,226.616828,279.579176,298.460267
4,518.831024,506.783591,493.942100,284.885691,246.415864
5,522.572861,510.507838,497.659757,271.726343,232.513237


In [23]:
# Otra manera de encontrar distancias entre dos puntos
from geopy.distance import distance
metros = distance((4.652834,-74.054339), (4.651420,-74.054638)).m
km = distance((4.652834,-74.054339), (4.651420,-74.054638)).km
print("Hay una distancia de " + str(metros) + " m")
print("Hay una distancia de " + str(km) + " km")

Hay una distancia de 159.84306824862307 m
Hay una distancia de 0.15984306824862307 km


<center>
<img src = "figs/dist1.png" alt = "coseno" style = "width: 500px;"/>
</center>

# Sistemas de Recomendación Basados en Ubicación

## Cercania

En este metodo vamos a buscar los restaurantes que se encuentren a cierta distancia de nosotros. Podemos ver que diferentes aplicaciones usan este metodo para encontrar de mostrar de manera simple nuestras opciones a nuestro alrededor. Por ejemplo, podemos ver como google maps nos recomienda restaurantes cercanos a nuestra ubicación

<center>
<img src = "figs/google_dist.png" style = "width: 700px;"/>
</center>

Para realizar estos sistemas de recomendación necesitaremos nuestra matriz de distancia entre usuarios y restaurantes que queramos recomendar, ademas necesitaremos de parametro a que usuario le queremos generar nuestra recomendación y la distancia maxima que esta persona estaria dispuesta a caminar (esta variable es un hiperparametro que puede variar mucho dependiendo de nuestos tipos de usuarios y la solución que queramos dar).

In [24]:
def cercania(usuario, metros):
    us = distancias.iloc[usuario-1]
    idx = us.index[us < metros]
    
    rest_cercanos = restaurants.loc[idx.tolist(),:].reset_index(drop = True)
    
    map = folium.Map(location = [4.65283,-74.054339], tiles = "OpenStreetMap", zoom_start = 16)
    # Otras opciones de tiles
    #Stamen Terrain, Toner, and Watercolor


    for i in range(0,len(rest_cercanos)):
        folium.Marker(
          location=[rest_cercanos.iloc[i]['latitud'], rest_cercanos.iloc[i]['longitud']],
           popup=rest_cercanos.iloc[i]['nombre'],
       ).add_to(map)

    folium.Marker(
          location=[usuarios.iloc[usuario-1]['latitud'], usuarios.iloc[usuario-1]['longitud']],
           popup=usuarios.iloc[usuario-1]['nombre'], icon=folium.Icon(color='red')
       ).add_to(map)
    
    return map

In [25]:
cercania(1,200)

Podemos observar en nuestro ejemplo que realizamos la recomendación al usuario Juan y pusimos que le mostraran los restaurantes que estuvieran a menos de 200m, ademas nuestra respuesta de la función es un mapa con la ubicación actual de Juan y nuestras recomendaciones de restaurantes con sus respectivos nombres.

En este caso podemos observar como Juan sus restaurantes mas cercanos son veganos y se encuentran relativamente cerca pero en esta recomendación solo le damos los restaurantes sin decirle cual podria gustarle mas o menos porque solo usamos la distancia entre los dos.


## Cercania con media de ratings

En este metodo realizaremos un sistema de recomendación de ubicación donde usaremas no solo la distancia sino la puntuación de estos restaurantes para poder recomendar los mejores x restaurantes mas cercanos para mostrarle a nuestro usuario, lo que podriamos modificar en cualquier momento y ver las diferentes de restaurantes que le recomendamos a cada persona.

<center>
<img src = "figs/google_ratings.jpeg" style = "width: 400px;"/>
</center>

Podemos observar en la imagen de arriba que ya no nos estan dando solo los restaurantes cercanos sino nos muestra primero el restaurante cercano con mayor puntuación.

In [26]:
def cercania_media(usuario, metros):
    # Buscamos a nuestro usuario con sus distancias a los restaurantes y dejamos solo los restaurantes que sean menores
    # a nuestro parametro de metros
    us = distancias.iloc[usuario-1]
    idx = us.index[us < metros]
    
    # Generamos un df que solo contenga estos restaurantes
    rest_cercanos = restaurants.loc[idx.tolist(),:].reset_index(drop = True)
    
    rest_cercanos = rest_cercanos.sort_values(by='rating', ascending=False)["nombre"][:2].tolist()
    
    return rest_cercanos

In [27]:
usuario = 1
recom = cercania_media(usuario,200)
recom

['Vegan Island ', 'El Pirata del sabor vegano']

In [28]:
# Filtramos solo los restaurantes recomendados de nuestro algoritmo
restaurants_recom = restaurants[restaurants['nombre'].isin(recom)].reset_index(drop=True)
restaurants_recom

,restaurant_id,latitud,longitud,nombre,tipo,rating,geometry
0,1,4.65142,-74.054638,Vegan Island,vegano,4.0,POINT (1002537.539 1006106.305)
1,3,4.65153,-74.054834,El Pirata del sabor vegano,vegano,4.0,POINT (1002515.791 1006118.468)


In [29]:
map = folium.Map(location = [4.65283,-74.054339], tiles = "OpenStreetMap", zoom_start = 16)
# Otras opciones de tiles
#Stamen Terrain, Toner, and Watercolor


for i in range(0,len(restaurants_recom)):
    folium.Marker(
      location=[restaurants_recom.iloc[i]['latitud'], restaurants_recom.iloc[i]['longitud']],
       popup=restaurants_recom.iloc[i]['nombre'],
   ).add_to(map)

folium.Marker(
      location=[usuarios.iloc[usuario-1]['latitud'], usuarios.iloc[usuario-1]['longitud']],
       popup=usuarios.iloc[usuario-1]['nombre'], icon=folium.Icon(color='red')
).add_to(map)
    

#Display el mapa
map

Podemos ver que ahora no nos recomendara todos los restaurantes que estan cercanos a nosotros, sino que tambien organizara de mayor a menor los restaurantes con mejor ranking y nos dejara los dos mejores restaurantes con un ranking de medias que en este caso serian Vegan Island y El Pirata del sabor vegano.

In [30]:
distancias[0]

0    159.843081
1    192.954635
2    529.994347
3    248.077661
4    518.831024
5    522.572861
Name: 0, dtype: float64

## Cercania con medias ponderadas

En este caso veremos un sistema de recomendación aun mas especifico donde no solo filtraremos por distancia sino que buscariamos los mejores con media ponderada que nos puede dar un resultado mucho mas especificas para cada usuario lo que mejorara el uso en este tipo de herramientas

In [31]:
# Veremos un repaso de nuestras matrices que realizamos en los pasos pasados sobre medias ponderadas
cosine_sim

user_id,1,2,3,4,5,6
user_id,,,,,,
1,1.000000,0.999630,0.997241,0.000000,0.000000,0.819533
2,0.999630,1.000000,0.998891,0.000000,0.000000,0.819836
3,0.997241,0.998891,1.000000,0.000000,0.000000,0.818927
4,0.000000,0.000000,0.000000,1.000000,1.000000,0.559888
5,0.000000,0.000000,0.000000,1.000000,1.000000,0.559888
6,0.819533,0.819836,0.818927,0.559888,0.559888,1.000000


In [32]:
r_matrix

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,NaN,NaN
2,4.0,3.0,4.0,NaN,NaN
3,3.0,2.0,3.0,NaN,NaN
4,NaN,NaN,NaN,4.0,5.0
5,NaN,NaN,NaN,4.0,5.0
6,4.0,3.0,4.0,2.0,4.0


In [33]:
r_matrix_dummy

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,0.0,0.0
2,4.0,3.0,4.0,0.0,0.0
3,3.0,2.0,3.0,0.0,0.0
4,0.0,0.0,0.0,4.0,5.0
5,0.0,0.0,0.0,4.0,5.0
6,4.0,3.0,4.0,2.0,4.0


In [34]:
def cercania_media_ponderada(usuario, metros):
    us = distancias.iloc[usuario-1]
    idx = us.index[us < metros]
    
    rest_cercanos = restaurants.loc[idx.tolist(),:].reset_index(drop = True)
    
    rankings = []
    for i in range(0,len(rest_cercanos)):
        rating = cf_user_wmean(usuario, rest_cercanos.iloc[i]["restaurant_id"])
        rankings.append(rating)
    
    rest_cercanos["rating_ponderado"] = rankings
    
    rest_cercanos = rest_cercanos[["restaurant_id","latitud", "longitud", "nombre", "rating_ponderado"]]
    
    rest_cercanos = rest_cercanos.sort_values(by='rating_ponderado', ascending=False)[:2]
    
    return rest_cercanos

In [35]:
usuario = 1
restaurants_recom = cercania_media_ponderada(usuario,200)
restaurants_recom

,restaurant_id,latitud,longitud,nombre,rating_ponderado
0,1,4.65142,-74.054638,Vegan Island,4.000723
2,3,4.65153,-74.054834,El Pirata del sabor vegano,4.000723


In [36]:
map = folium.Map(location = [4.65283,-74.054339], tiles = "OpenStreetMap", zoom_start = 16)
# Otras opciones de tiles
#Stamen Terrain, Toner, and Watercolor


for i in range(0,len(restaurants_recom)):
    folium.Marker(
      location=[restaurants_recom.iloc[i]['latitud'], restaurants_recom.iloc[i]['longitud']],
       popup=restaurants_recom.iloc[i]['nombre'],
   ).add_to(map)

folium.Marker(
      location=[usuarios.iloc[usuario-1]['latitud'], usuarios.iloc[usuario-1]['longitud']],
       popup=usuarios.iloc[usuario-1]['nombre'], icon=folium.Icon(color='red')
).add_to(map)
    

#Display el mapa
map

Podemos observar que en este sistema de recomendación logramos ubicar los restaurantes mas cercanos a nuestro usuario y le hicimos una recomnedación personalizada dependiendo de sus gustos cuales son los restaurantes cercanos que con mayor posiblidad le gusten.

## Cercania con medias ponderadas y tipo de comida

En este siguiente modelo de recomendación observaremos como usamos lo visto en el anterior pero le agregaremos un filtro de categoria por tipo de restaurante, esto lo haremos porque muchas veces una persona quiere que le recomendemos un restaurante que este cerca, que seguramente le guste pero tal vez quiere un tipo de comida en especifico o un rango de precio y para esto entra en juego aprender a usar filtros para poder personalizar de mejor manera la busqueda de nuestros usuarios en nuestra plataforma y mas cuando hay muchos productos y servicios que solo pueden confundir y no dejarlo decidir por la gran cantidad.

In [37]:
restaurants

,restaurant_id,latitud,longitud,nombre,tipo,rating,geometry
0,1,4.651420,-74.054638,Vegan Island,vegano,4.000000,POINT (1002537.539 1006106.305)
1,2,4.651475,-74.054732,La Verdura Inquieta,vegano,3.000000,POINT (1002527.109 1006112.387)
2,3,4.651530,-74.054834,El Pirata del sabor vegano,vegano,4.000000,POINT (1002515.791 1006118.468)
3,4,4.655471,-74.057151,Muuu said the happy cow,carne,3.333333,POINT (1002258.695 1006554.262)
4,5,4.655363,-74.057581,Paul's Mcarne Feast Palace,carne,4.666667,POINT (1002210.985 1006542.318)


In [38]:
usuarios

,user_id,nombre,latitud,longitud,geometry
0,1,Juan,4.652834,-74.054339,POINT (1002570.709 1006262.668)
1,2,Maria,4.653065,-74.054058,POINT (1002601.887 1006288.214)
2,3,Pablo,4.653195,-74.059075,POINT (1002045.225 1006302.573)
3,4,Joaquin,4.653328,-74.055814,POINT (1002407.049 1006317.291)
4,5,Martina,4.653515,-74.058822,POINT (1002073.295 1006337.960)
5,6,Rosario,4.653647,-74.058792,POINT (1002076.624 1006352.557)


In [39]:
def cercania_media_ponderada_tipo(usuario, metros, tipo):
    us = distancias.iloc[usuario-1]
    idx = us.index[us < metros]
    
    rest_cercanos = restaurants.loc[idx.tolist(),:].reset_index(drop = True)
    
    rest_cercanos = rest_cercanos[rest_cercanos["tipo"] == tipo].reset_index(drop = True)
    
    rankings = []
    for i in range(0,len(rest_cercanos)):
        rating = cf_user_wmean(usuario, rest_cercanos.iloc[i]["restaurant_id"])
        rankings.append(rating)
    
    rest_cercanos["rating_ponderado"] = rankings
    
    rest_cercanos = rest_cercanos[["restaurant_id","latitud", "longitud", "nombre", "rating_ponderado"]]
    
    rest_cercanos = rest_cercanos.sort_values(by='rating_ponderado', ascending=False)[:2]
    
    return rest_cercanos

In [40]:
usuario = 4
restaurants_recom = cercania_media_ponderada_tipo(usuario, 500, "vegano")
restaurants_recom

,restaurant_id,latitud,longitud,nombre,rating_ponderado
0,1,4.65142,-74.054638,Vegan Island,4.0
2,3,4.65153,-74.054834,El Pirata del sabor vegano,4.0


In [41]:
map = folium.Map(location = [4.65283,-74.054339], tiles = "OpenStreetMap", zoom_start = 16)
# Otras opciones de tiles
#Stamen Terrain, Toner, and Watercolor


for i in range(0,len(restaurants_recom)):
    folium.Marker(
      location=[restaurants_recom.iloc[i]['latitud'], restaurants_recom.iloc[i]['longitud']],
       popup=restaurants_recom.iloc[i]['nombre'],
   ).add_to(map)

folium.Marker(
      location=[usuarios.iloc[usuario-1]['latitud'], usuarios.iloc[usuario-1]['longitud']],
       popup=usuarios.iloc[usuario-1]['nombre'], icon=folium.Icon(color='red')
).add_to(map)
    

#Display el mapa
map

En este caso logramos observar como implementamos un sistema de recomendación con varios metodos que nos van a permitir mejorar la personalización de nuestra aplicación con nuestro usuario y como esto permitira que encuentre los mejores porductos de su gusto de una manera sencilla pero ahora entendemos todo lo que tiene detras estos metodos y porque nos serviran en mejorar la retención de tiempo de uso o de pedidos en nuestros servicios.

## Analisis de Sentimientos y Modelos Ya Entrenados

Volveremos a lo aprendido en clases pasadas sobre analisis de texto y su analisis de sentimiento pero veremos el uso de [Hugging Face](https://huggingface.co/) para buscar modelos ya entrenados que podremos usar con nuestras bases de datos sin tener que entrenar un modelo desde cero y solo tendremos que tener nuestro input listo y pasarlo por el modelo.
¿Que es?  Esta página es como un almacenamiento global de módulos de deep learning, sobre todo de aquellos aplicados a temas de lenguaje. Estos modelos están hechos por las empresas más reconocidas a nivel mundial, entre ellas Google, Microsoft o Amazon.

Así pues, todos los data scientists que trabajan en las empresas publican sus modelos de tecnología punta o de las últimas arquitecturas; todo está allí, dentro de Hugging Face, de modo que la gente puede utilizar lo que necesite, mirar cómo trabajan y aprender de los mejores.

Por otro lado, en esta clase lo usaremos para usar [Beto](https://huggingface.co/dccuchile/bert-base-spanish-wwm-cased) que es el modelo de lenguaje publico con mayor entrenamiento que hay ya que es open source y lo usaremos para hacer analisis de sentimientos

In [42]:
#pip install pysentimiento

In [43]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

In [44]:
analyzer.predict("Qué gran jugador es Messi")

AnalyzerOutput(output=POS, probas={POS: 0.946, NEU: 0.037, NEG: 0.017})

In [45]:
analyzer.predict("yayyy")

AnalyzerOutput(output=POS, probas={POS: 0.472, NEU: 0.460, NEG: 0.068})

In [46]:
analyzer.predict("Esto es pésimo")

AnalyzerOutput(output=NEG, probas={NEG: 0.887, NEU: 0.098, POS: 0.014})

In [47]:
ratings

,restaurant_id,user_id,rating,comentarios
0,1,1,5.0,Excelente Servicio. La comida exquisita. Mejor...
1,1,2,4.0,"La comida deliciosa, el trato puede mejorar"
2,1,3,3.0,"Muy buena comida, pero mal servicio"
3,1,4,NaN,NaN
4,1,5,NaN,NaN
5,1,6,4.0,NaN
6,2,1,4.0,"Buen menu, poca variedad"
7,2,2,3.0,Buena comida
8,2,3,2.0,Comida mediocre y mal servicio
9,2,4,NaN,NaN


In [48]:
sentimiento =[]
for i in range (0,len(ratings)):
    if pd.isna(ratings.iloc[i]["comentarios"]):
        sentimiento.append("")
    else:
        sent = analyzer.predict(ratings.iloc[i]["comentarios"])
        sentimiento.append(sent)
ratings["Sentimientos"] = sentimiento
ratings

,restaurant_id,user_id,rating,comentarios,Sentimientos
0,1,1,5.0,Excelente Servicio. La comida exquisita. Mejor...,"AnalyzerOutput(output=POS, probas={POS: 0.965,..."
1,1,2,4.0,"La comida deliciosa, el trato puede mejorar","AnalyzerOutput(output=POS, probas={POS: 0.864,..."
2,1,3,3.0,"Muy buena comida, pero mal servicio","AnalyzerOutput(output=NEG, probas={NEG: 0.808,..."
3,1,4,NaN,NaN,
4,1,5,NaN,NaN,
5,1,6,4.0,NaN,
6,2,1,4.0,"Buen menu, poca variedad","AnalyzerOutput(output=NEG, probas={NEG: 0.633,..."
7,2,2,3.0,Buena comida,"AnalyzerOutput(output=NEU, probas={NEU: 0.503,..."
8,2,3,2.0,Comida mediocre y mal servicio,"AnalyzerOutput(output=NEG, probas={NEG: 0.977,..."
9,2,4,NaN,NaN,


In [49]:
print(ratings.iloc[0]["comentarios"])
print(ratings.iloc[0]["Sentimientos"])
print(ratings.iloc[0]["rating"])

Excelente Servicio. La comida exquisita. Mejor restaurante vegano!
AnalyzerOutput(output=POS, probas={POS: 0.965, NEU: 0.031, NEG: 0.004})
5.0


In [50]:
print(ratings.iloc[1]["comentarios"])
print(ratings.iloc[1]["Sentimientos"])
print(ratings.iloc[1]["rating"])

La comida deliciosa, el trato puede mejorar
AnalyzerOutput(output=POS, probas={POS: 0.864, NEU: 0.131, NEG: 0.005})
4.0


In [51]:
print(ratings.iloc[8]["comentarios"])
print(ratings.iloc[8]["Sentimientos"])
print(ratings.iloc[8]["rating"])

Comida mediocre y mal servicio
AnalyzerOutput(output=NEG, probas={NEG: 0.977, NEU: 0.019, POS: 0.004})
2.0


# Evaluación Sistemas de Recomendación


- La evaluación de los sistemas de recomendación es otro paso importante para evaluar la efectividad del método. 


- Cuando se trata de etiquetas numéricas, como las calificaciones de 5 estrellas, la forma más común de validar un sistema de recomendación se basa en su valor de predicción, es decir, la capacidad de predecir las elecciones del usuario. Las funciones estándar como el error cuadrático medio (RMSE), la precisión, las curvas ROC, etc...


- Sin embargo, hay varias otras formas de evaluar los sistemas. Esto sucede porque distintos actores pueden tener distintos intereses.


- Imaginemos las siguientes tres personas: 

    (a) un analista de marketing
    
    (b) un programador
    
    (c) un usuario final
    
    
- Está claro que lo relevante para todos ellos no es lo mismo. 

    - para el analista de marketing, lo que suele ser importante es cómo el sistema ayuda a impulsar el producto, 
    - para el programador es qué tan eficiente es el algoritmo, 
    - para el usuario final es si el sistema le da buenos resultados o, en general, buenos. 
    
    
 - En la literatura podemos ver dos tipologías principales: evaluación offline y online.

    - La evaluación offline es la que estamos acostumbrados en ML supervizado donde usamos un conjunto de entrenamiento y un conjunto de prueba; y aplicamos la métrica de evaluación apropiada. 
    - La evaluación online es cuando utilizamo un conjunto de herramientas que nos permite observar las interacciones de los usuarios con el sistema. 
        - La técnica en línea más común se llama prueba A-B y tiene la ventaja de permitir la evaluación del sistema al mismo tiempo que los usuarios aprenden, compran o juegan con el sistema de recomendación. 
        - Esto acerca la evaluación al funcionamiento real del sistema y la hace realmente efectiva cuando el propósito del sistema es cambiar o influir en el comportamiento de los usuarios. 
        - Para evaluar la prueba, estamos interesados en medir cómo cambia el comportamiento del usuario cuando el usuario interactúa con diferentes sistemas de recomendación. Y require estimación de parámetros causales.
        - Pongamos un ejemplo: imagina que queremos desarrollar un sistema de recomendación de música como Spotify, donde tu unico objetivo final es que los usuarios solo escuchen una estación de música inteligente y pasen mucho tiempo escuchándola. 
        - En esta situación, las métricas fuera de línea como RMSE no son lo suficientemente buenas. En este caso, estamos particularmente interesados en la evaluación del objetivo global del sistema de recomendación, la retención de usuario.